In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
from pathlib import Path

In [23]:
DATA_FOLDER = Path("fund_data")   


excel_files = [
     "aditya birla index fund.xlsx",
    "hdfc index fund.xlsx",
    "icici nifty 50.xlsx",
    "sbi index fund.xlsx",
    "uti index fund.xlsx"
]


display_names = {
    "aditya birla index fund.xlsx":         "Aditya Birla Nifty Index Fund",
    "hdfc index fund.xlsx":             "HDFC Index Fund",
    "icici nifty 50.xlsx":           "ICICI Nifty Fund",
    "sbi index fund.xlsx":       "SBI Index Fund",
    "uti index fund.xlsx":    "UTI Index fund"
}

In [24]:
all_funds = []

for file_name in excel_files:
    file_path = DATA_FOLDER / file_name
    
    if not file_path.exists():
        print(f"NOT FOUND → {file_path}")
        continue
    
    # Read Excel 
    df = pd.read_excel(file_path, header=4)
    
    
    df = df[["Date", "Net Asset Value"]].copy()
    
    # Clean data
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df["Net Asset Value"] = pd.to_numeric(df["Net Asset Value"], errors="coerce")
    df = df.dropna(subset=["Date", "Net Asset Value"])
    
    # Add clean Fund Name
    fund_display_name = display_names.get(file_name, file_name.replace(".xlsx", ""))
    df["Fund Name"] = fund_display_name
    
    
    df = df[["Fund Name", "Date", "Net Asset Value"]]
    df.rename(columns={"Net Asset Value": "NAV"}, inplace=True)
    
    all_funds.append(df)
    print(f"Loaded → {fund_display_name:<40} | {len(df):,} records")

# Combine all funds
master_df = pd.concat(all_funds, ignore_index=True)
master_df = master_df.sort_values(["Fund Name", "Date"]).reset_index(drop=True)

print(f"\nTOTAL RECORDS: {len(master_df):,}")
print(f"Date range : {master_df['Date'].min().date()} → {master_df['Date'].max().date()}\n")
print("="*95)

Loaded → Aditya Birla Nifty Index Fund            | 986 records
Loaded → HDFC Index Fund                          | 986 records
Loaded → ICICI Nifty Fund                         | 986 records
Loaded → SBI Index Fund                           | 991 records
Loaded → UTI Index fund                           | 986 records

TOTAL RECORDS: 4,935
Date range : 2021-11-16 → 2025-11-14



In [20]:
# Step 1: Check if cagr_results is really empty 
print("cagr_results contains:", cagr_results)
print("Length:", len(cagr_results))

cagr_results contains: []
Length: 0


In [25]:


cagr_results = []

for fund in master_df["Fund Name"].unique():
    temp = master_df[master_df["Fund Name"] == fund].sort_values("Date")
    
    if len(temp) < 2:
        continue
        
    start_nav = temp.iloc[0]["NAV"]
    end_nav   = temp.iloc[-1]["NAV"]
    start_date = temp.iloc[0]["Date"]
    end_date   = temp.iloc[-1]["Date"]
    years = (end_date - start_date).days / 365.25
    
    
    if years < 1:  #  skip if less than 1 year
        continue
        
    cagr_pct = (end_nav / start_nav) ** (1/years) - 1
    
    cagr_results.append({
        "Fund Name": fund,
        "Period (Years)": round(years, 2),
        "CAGR (%)": round(cagr_pct * 100, 2)
    })


cagr_df = pd.DataFrame(cagr_results).sort_values("CAGR (%)", ascending=False).reset_index(drop=True)
cagr_df

,Fund Name,Period (Years),CAGR (%)
0,Aditya Birla Nifty Index Fund,3.99,13.95
1,UTI Index fund,3.99,10.62
2,SBI Index Fund,3.99,10.60
3,HDFC Index Fund,3.99,10.40
4,ICICI Nifty Fund,3.99,10.38


In [26]:


print("="*80)
print("MUTUAL FUND 7-YEAR CAGR ANALYSIS RESULT".center(80))
print("(Data available only from Nov-2021 → ~4 years CAGR shown)".center(80))
print("="*80)


cagr_df = cagr_df.sort_values("CAGR (%)", ascending=False).reset_index(drop=True)

print("\nTOP 2 PERFORMING MUTUAL FUNDS")
print("-" * 50)
for i in range(min(2, len(cagr_df))):
    row = cagr_df.iloc[i]
    print(f"{i+1}. {row['Fund Name']:<25} → {row['CAGR (%)']:6.2f}%")

print("\nWORST 2 PERFORMING MUTUAL FUNDS")
print("-" * 50)
for i in range(max(0, len(cagr_df)-2), len(cagr_df)):
    row = cagr_df.iloc[i]
    print(f"{i+1}. {row['Fund Name']:<25} → {row['CAGR (%)']:6.2f}%")

# NAV Swing Detection > ±5%
print("\n" + "="*80)
print("NAV SWING DETECTION (> ±5%)".center(80))
print("="*80)

swings = []
for fund in master_df["Fund Name"].unique():
    df_fund = master_df[master_df["Fund Name"] == fund].sort_values("Date").reset_index(drop=True)
    for i in range(1, len(df_fund)):
        prev = df_fund.loc[i-1, "NAV"]
        curr = df_fund.loc[i, "NAV"]
        pct_change = (curr - prev) / prev * 100
        if abs(pct_change) > 5:
            swings.append({
                "Fund Name": fund,
                "Date": df_fund.loc[i, "Date"].strftime("%d-%b-%Y"),
                "% Change": f"{pct_change:+.2f}%"
            })

if swings:
    swings_df = pd.DataFrame(swings)
    print(swings_df.to_string(index=False))
    print(f"\nTotal days with > ±5% change: {len(swings)}")
else:
    print("No NAV swing greater than ±5% found in any fund.")

print("\n" + "="*80)
print("ALL TASKS COMPLETED SUCCESSFULLY!")
print("You can now submit this output.")
print("="*80)

                    MUTUAL FUND 7-YEAR CAGR ANALYSIS RESULT                     
           (Data available only from Nov-2021 → ~4 years CAGR shown)            

TOP 2 PERFORMING MUTUAL FUNDS
--------------------------------------------------
1. Aditya Birla Nifty Index Fund →  13.95%
2. UTI Index fund            →  10.62%

WORST 2 PERFORMING MUTUAL FUNDS
--------------------------------------------------
4. HDFC Index Fund           →  10.40%
5. ICICI Nifty Fund          →  10.38%

                          NAV SWING DETECTION (> ±5%)                           
                    Fund Name        Date % Change
Aditya Birla Nifty Index Fund 04-Jun-2024   -5.48%
              HDFC Index Fund 04-Jun-2024   -5.85%
             ICICI Nifty Fund 04-Jun-2024   -5.85%
               SBI Index Fund 04-Jun-2024   -5.87%
               UTI Index fund 04-Jun-2024   -5.85%

Total days with > ±5% change: 5

ALL TASKS COMPLETED SUCCESSFULLY!
You can now submit this output.
